# TP Mésurer la consommation énergétique d'un ressource de calcul
Auteur : Danilo Carastan dos Santos et Mathilde Jay    
Date : 02-10-2023

### Objectifs
- Utiliser des outils de monitoring d’énergie
- Visualiser quels composants sont inclus dans les outils de mésure
- Étudier l’impact du nombre de ressources dans la consommation énergétique

### Préambule
Tous les outils de mesure d'énergie nécessitent soit des droits de superutilisateur sur la machine, soit du matériel supplémentaire (wattmètre). Les fournisseurs de cloud public standard (AWS, Google Cloud, etc.) ne vous donnent pas accès à de tels outils. Néanmoins, le contenu présenté dans ce TP devrait être utile lors de (i) la phase de développement, où l'application s'exécute sur un ordinateur local, ou (ii) dans un cloud privé, où un contrôle plus approfondi du système/matériel est plus possible.

## Partie 0 : Prendre en main Grid'5000
### Description de la plateforme
Grid’5000 est un banc de test ("testbed" en anglais). Il s’agit d’une plate-forme expérimentale informatique
qui permet de réaliser des tests rigoureux sur des outils de calcul numérique haute performance, notamment en
calcul parallèle et distribué.   
Grid’5000 s’organise de la manière suivante :
- 8 sites en France
- 800 noeuds localisés à Grenoble (92), Lille (39), Luxembourg (16), Lyon (132), Nancy (189), Nantes (70),
Rennes (173), Sophia (44)
- environ 15000 coeurs au total

Vous pourrez accéder à toutes les caractéristiques techniques du matériel ici : https://www.grid5000.fr/w/Hardware.       
L'utilisation de la plateforme est soumis à des régles simples de partages, qui sont détaillées ici : https://www.grid5000.fr/w/Grid'5000:UsagePolicy.    
Les réservations des ressources passe par OAR qui est un utilitaire développé par Inria et utilisé pour le calcul haute performance. Les jobs sont définis par un JOB_ID.   

### Création du compte
Vous avez dû recevoir un email dans le semaine avec les informations nécessaires pour accéder à la plateforme. Il y a deux étapes à faire : Ouvrir une URL et ajouter un mot de passe et une clef ssh.

Pour générez une clef ssh :   
`ssh-keygen -t rsa`   
qui crée deux fichiers dans le dossier caché `./ssh`. id_rsa est la clef privée (à garder secrète)
et id_rsa.pub la clef publique que vous transmettrez à Grid’5000.

Ces étapes sont détaillées ici : https://www.grid5000.fr/w/Tutorial_or_Teaching_Labs_Trainee_HowTo.

### Accéder à la plateforme

Pour ce TP, on vous propose d'utiliser l'interface Jupyter de Grid'5000, ce qui va vous permettre de visualiser l'évolution de l'énergie plus facilement : https://intranet.grid5000.fr/notebooks/. Suivez les étapes suivantes :
- Créer un nouveau serveur
- Sélectionner le site de Lyon
- Cliquer sur "reserved node"
- Container ID : `CONTAINER_ID`
- Walltime : 1:25
- Cliquer sur start

`CONTAINER_ID` est un chiffre que nous vous donnerons lors du début du TP

![Alt text](../Figures/g5k_create_jupyter_server.png)


### Télécharger le contenu du TP
Dans l'interface d'accueil de JupyterLab, ouvrez un nouveau terminal

![Alt text](../Figures/jupyterlab_terminal_button.png)

Puis, lancez la commande suivante 

```
git clone https://github.com/danilo-carastan-santos/measuring-energy-grid5000.git
```

Et ouvrez le Jupyter Notebook de ce TP (ce fichier, `./Notebooks/Lab_measuring_energy_g5k_FR.ipynb`)


### Créer l'environnement vituel

Dans ce même terminal lancez les commandes suivantes

```
python -m venv venv/
source venv/bin/activate
pip install ipykernel
./venv/bin/python -m ipykernel install --user --name TP_conso --display-name "Python (TP_conso)"
pip install pandas matplotlib requests numpy seaborn
```

### Ouvrir le notebook du TP

Vous pouvez ouvrir les notebooks un outilisant le navigateur de JupyterLab (côté gauche de l'interface).

Ouvrez le notebook, et selectionnez le kernel que vous venez de créer (`TP_Conso`). 

![Alt text](../Figures/change_ipykernel_1.png)
![Alt text](../Figures/change_ipykernel_2.png)

## Partie 1 : Comparer plusieurs méthodes de mésure

Dans cette partie, vous allez comparer trois méthodes de mésure énergétique: (i) par TDP, (ii) par logiciel (perf/RAPL) et (iii) par wattmètre phisique.

### Méthode par TDP (Thermal Design Power)

Pour cette approche, il suffit de connaître le TDP du processeur utilisé et le temps d'éxecution de l'application. 

Pour le premier, vous pouvez obtenir d'informations sur le processeur avec la commande 

```
cat /proc/cpuinfo
```

**Question 1:** Pourquoi la commande dessus semble afficher plusieurs fois la même information?

*Double-cliquez ici pour rédiger votre réponse.*

**Question 2:** En cherchant sur Internet, quel est le TDP du processeur utilisé?

*Double-cliquez ici pour rédiger votre réponse.*

### Wattmètre physique de Grid'5000
Les machines nova sont monitorées en permanance par un wattmètre physique.     
Les données du wattmètre sont collectées et exposées par Kwollect. Kwollect est le service de monitoring
disponible dans Grid’5000. Il collecte et met à disposition les métriques environnementales et de performance
des nœuds. Vous pouvez trouver plus de détails sur le monitoring de Grid’5000 avec Kollect ici : https://www.grid5000.fr/w/Monitoring_Using_Kwollect.
Vous pouvez visualiser les métriques collectées par Kwollect de deux manières principales : via les tableaux de
bord Grafana et en consultant directement l’API Kwollect.


*Grafana*    
Grid’5000 met a disposition les tableaux de bord Grafana afin de visualiser les métriques disponibles dans
Kwollect pour l’ensemble des noeuds. Vous pouvez consulter les tableau de bord du site Lyon ici : https://api.grid5000.fr/stable/sites/lyon/metrics/dashboard/d/kwollect/kwollect-metrics?var-device_id=nova-9&var-metric_id=wattmetre_power_watt&orgId=1. Pour visualiser
les données des wattmètres physiques vous devez sélectionner le noeud dans l’onglet device et la métrique
wattmetre_power_watt.


*API*    
Kwollect met a disposition l’ensemble des métriques via un API web. Vous pouvez récupérer les données de
consommation d’un noeud directement depuis l’API.      
https://api.grid5000.fr/stable/sites/lyon/metrics?nodes=nova-1&metrics=wattmetre_power_watt&start_time=2022-09-21T12:00:00&end_time=2022-09-21T12:10:00    
Ouvrez ce lien avec votre navigateur web. En suivant ce lien, vous récupérez les données
renvoyées par le wattmètre physique installé sur le nœud nova-1 à Lyon pour la période de
12h00 à 12h10 le 21 septembre 2022. 

Pour ce TP, un script a été écrit pour récupérer les valeurs d'énergie entre deux timestamps donnés. Le code se trouve dans le fichier `utils.py` et en particulier dans la fonction `retrieve_power()` de l'objet `Wattmeter`. Les autres fonctions traitent les données pour pouvoir récupérer directement un dataframe.

**Question 3:** À quelle fréquence Kwollect renvoie-t-il la métrique wattmetre_power_watt ?

*Double-cliquez ici pour rédiger votre réponse.*

### Wattmètre logiciel (RAPL/Perf)
Pour ce TP, vous allez utiliser `perf` de la manière suivante : 

`sudo perf stat -A -a -e 'power/energy-ram/' -e 'power/energy-pkg/' -o FILE -x ";" -I INTERVAL_TIME_MS`   

Par exemple :  

`sudo perf stat -A -a -e 'power/energy-ram/' -e 'power/energy-pkg/' -o /tmp/result_ep.csv -x ";" -I 100`  

L'outil `perf` permet de surveiller plusieurs événements de performance de la machine. Vous pouvez répertorier ces événements en utilisant la commande `perf list`. Pour ce TP, nous nous intéressons à deux événements, `power/energy-ram/` et `power/energy-pkg/`, qui contiennent des informations fournies par l'interface RAPL. `power/energy-ram/` surveille la consommation d'énergie de la mémoire, et `power/energy-pkg/` surveille la consommation d'énergie du socket CPU.

Pour une utilisation basique, la commande est à lancer *en parallèle dans le terminal* avant chaque execution, et à interrompre après la fin de chaque execution. Attendez quelques secondes avant et après pour être sûr de visualiser toute l'execution. L'énergie est monitorée en temps réel et les résultats sont enregistrés dans un fichier CSV (FILE) quand vous interrompez la commande. Veillez à changer le nom du fichier selon le nom de l'exercice pour pouvoir tout garder en mémoire.

**Question 4:** À quoi servent les paramètres `-A` et `-a` de la commande `perf stat`?
Indice: Pensez a lancer la commande `perf stat --help`

*Double-cliquez ici pour rédiger votre réponse.*

### Lancer les mésures

Lancez chaque célule ci-dessous.

**À noter:** Nous avons besoin de droits superutilizateur (i.e., `sudo`) pour collecter les données Perf/RAPL.
Cela est dû à un risque de sécurité: https://www.intel.com/content/www/us/en/security-center/advisory/intel-sa-00389.html
Grid5000 nous donne de droits superutilizateur par la commande `sudo-g5k`

In [ ]:
!hostname

Si vous trouvez cet erreur lors de l'exécution de la commande dessus

```
/var/lib/oar/.batch_job_bashrc: line 5: /home/username/.bashrc: No such file or directory
nova-XX.lyon.grid5000.fr
```

Il faut juste lancer la commande `touch .bashrc` dans le terminal.

In [ ]:
import os
import socket

NOEUD = socket.gethostname().rstrip(".lyon.grid5000.fr")
HOME_DIR = os.getenv('HOME')

REPOSITORY_DIR = HOME_DIR + "/"
TP_DIR = REPOSITORY_DIR + "TP_conso/"
PERF_RESULTS = TP_DIR + "perf_results/"

!mkdir $TP_DIR
!mkdir $PERF_RESULTS
!sudo-g5k
results = {}

In [ ]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import random
import sys
import time
debut_tp = time.time()
sys.path.append(REPOSITORY_DIR)

#TODO: for the future, improve packaging
from utils import get_perf_df, qsort, insertion_sort, Wattmeter, get_data, plot_timeseries

## Test de mésure avec un stress de CPU

Pour commencer, lançons un stress sur le CPU. Vous pouvez faire varier le nombre de workers avec l'argument `-c`.    

Pour éviter d'utiliser la commande `perf stat` en parallèle dans un terminal (usage basique), nous allons utiliser la librarie Python `subprocess` qui permet lancer des commandes depuis un code Python

In [ ]:
import subprocess
import os

NB_CPUS = 1
PROC_TIME_SECONDS = 10
SLEEP_TIME_SECONDS = 1

PERF_RESULT_FILENAME = "CPU.csv"
PERF_RESULT_CSV_FILEPATH = PERF_RESULTS + PERF_RESULT_FILENAME

perf_command = ["sudo","perf","stat",
                "-A","-a",
                "-e","'power/energy-ram/'",
                "-e","'power/energy-pkg/'",
                "-x",";",
                "-I", "100"]

stress_command = ["stress","-c",str(NB_CPUS),"-t",str(PROC_TIME_SECONDS)]

with open(PERF_RESULT_CSV_FILEPATH, mode="w") as output_file:

    perf_start = time.time()
    ## Lancer perf stat et ne pas attendre sa fin
    perf_pid = subprocess.Popen(perf_command, stdout=output_file, stderr=output_file).pid
    
    ## Attendre un peu pour bien monitorer la commande stress
    time.sleep(SLEEP_TIME_SECONDS)    

    stress_start = time.time()
    ## Lancer la commande stress et attendre sa fin
    subprocess.run(stress_command)
    stress_end = time.time()

    ## Attendre un peu pour bien monitorer la commande stress
    time.sleep(SLEEP_TIME_SECONDS)

    ## Terminer la commande perf stat en envoyant SIGINT (-2)
    ## Équivalent à Ctrl+c
    subprocess.run(["kill", "-2", str(perf_pid)])
    print("Done")


### Collecte et sauvegarde de résultats

In [ ]:
name=PERF_RESULT_FILENAME.rstrip(".csv")
results[name] = {}
results[name]["stress_start"]=stress_start
results[name]["stress_end"]=stress_end
results[name]["perf_start"]=perf_start

perf_energy_joules, perf_df, watt, watt_df = get_data(name, results, NOEUD)
perf_df_plot = perf_df[
        (perf_df["timestamp"]>results[name]["stress_start"]-5)&(perf_df["timestamp"]<results[name]["stress_end"]+5)]

RESULT_COLS = ["nb_cpus", "tool", "energy_joule"]
result = [{"nb_cpus": NB_CPUS,
          "tool": "perf",
          "energy_joule": perf_energy_joules},
          {"nb_cpus": NB_CPUS,
          "tool": "wattmetre",
          "energy_joule": watt.results['wattmetre_power_watt']["energy_joule"]}
         ]

results[name]["perf_energy_joules"]=perf_energy_joules
results[name]["wattmetre_energy_joules"]=watt.results['wattmetre_power_watt']["energy_joule"]
results[name]["diff_energy_joules"]=watt.results['wattmetre_power_watt']["energy_joule"]- perf_energy_joules

df_result = pd.DataFrame(result)
print(df_result)

RESULT_FILENAME = "stress-nb_cpus="+str(NB_CPUS)+"-tstamp="+str(int(stress_start))+".csv"
df_result.to_csv(PERF_RESULTS+RESULT_FILENAME, index=False)
print("Résultats sauvegardés dans le fichier", PERF_RESULTS+RESULT_FILENAME)

**Question 5:** Utilisez la formule TDP pour estimer la consommation énergétique de la commande `stress`. Utiliser la valeur de `PROC_TIME_SECONDS` pour le temps d'exécution. Que remarquez-vous par rapport à l'estimation TDP et aux mésures de `perf` et `wattmetre` ? 

*Double-cliquez ici pour rédiger votre réponse.*

### Vlisualisation des profils énergétiques

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax = plot_timeseries(ax, perf_df_plot, watt_df, name)

**Question 6:** Quelles sont les différences entre le wattmètre physique et le wattmètre logiciel ?

*Double-cliquez ici pour rédiger votre réponse.*

### Consommation énergétique en fonction du nombre de CPUs

**Exercice 1:** Relancez les parties "Test de mésure avec un stress de CPU" et "Collecte et sauvegarde de résultats" plusieurs fois, chaque fois en modifiant la variable `NB_CPUS` par `1`, `2`, `4`, `8`, `16`, `32` (cinq relances au total). Utilisez les fichiers CSV sauvegardés (exemple `/home/username/TP_conso/perf_results/stress-nb_cpus=XX-tstamp=YYYYYYY.csv` ou `XX` est le nombre de CPUs) pour faire une figure (c.f., méthode `seaborn.lineplot()`) qui illustre l'évolution de la consommation énergétique totale en fonction du nombre de CPUs par chaque métode de mésure (sauf TDP).

In [ ]:
## double-cliquez ici pour coder

**Question 7:** Que remarquez-vous par rapport à votre figure ?

*Double-cliquez ici pour rédiger votre réponse.*

## Partie 2: Ouverture vers les émissions carbones
Utilisez la celulle suivante pour mesurer l'énergie que le noeud que vous utilisez a consommé durant le TP. 
**À combien de CO2 équivalent cela corresponds ? À combien d'euros ? Vous trouverez sur internet des valeurs moyennes de coût et d'intensité carbone de l'electricité en France. Extrapoler à une journée, un mois, un année, ...**
Le site https://app.electricitymaps.com/zone/FR?wind=true donne l'évolution de l'intensité carbone par pays en live. **Comment est-ce que l'intensité carbone varie en France ou dans un autre pays ? Comment est ce que cela impacte les émissions carbone de ce TP ?** 

In [ ]:
fin_tp = time.time()
get_data(name, results, NOEUD)
print("Done")

In [ ]:
watt = Wattmeter(
        NOEUD, 
        "lyon", 
        debut_tp, 
        fin_tp, 
        metrics=["wattmetre_power_watt"])
print("Énergie consommée pendant tous le TP :", watt.results['wattmetre_power_watt']["energy_joule"], "joules")

*Double-cliquez ici pour rédiger votre réponse.*